# yes24 크롤링

In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
import time
import random


from fake_useragent import UserAgent

In [25]:
def AladinBookListCrawler(CategoryNum, year_start, year_end, month, page):

    url_1 = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=MonthlyBest&BranchType=1&CID='
    url_2 = '&Year='
    url_3 = '&Month='
    url_4 = '&Week=3&page=' # 월간 판매량은 week3 기준
    url_5 = '&cnt=1000&SortOrder=1'

    book_list = []

    for i in range(year_start, year_end+1) :
        
        rank=0
        
        print("Crawling 시작!")
        print(f"====================>> {i}년")
        
        for j in range(1,page+1):
            
            print(f"====={j}페이지=====")
            
            ua = UserAgent()
            url_set = url_1 + str(CategoryNum) + url_2 + str(i) + url_3 + str(month) + url_4 + str(j) + url_5
            headers = {'User-Agent' : ua.random}
            res = requests.get(url_set, headers=headers)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "lxml")
            
            books = soup.find_all('a', class_='bo3')
            
            for book in books:
                time.sleep(random.uniform(5,8) )
                
                rank += 1 # 책 순위
                
                book_url = book["href"]
                sub_res = requests.get(book_url)
                sub_res.raise_for_status()
                soup = BeautifulSoup(sub_res.text, "lxml")
                
                try :
                    book_info = soup.find("script")
                    json_objects = json.loads(book_info.contents[0])
                    
                    book_name = json_objects['name']
                    
                    print(f"====={rank}위, {book_name}=====")
                    
                    author = json_objects['author']['name']
                    author = author.replace(",", " /")
                    pub_by = json_objects['publisher']['name']
                    pub_day = json_objects['workExample'][0]['datePublished']
                    price = int(json_objects['workExample'][0]['potentialAction']['expectsAcceptanceOf']['Price'])
                    genre1 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(2)').get_text()
                    genre2 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(3)').get_text()
                    genre3 = soup.select_one('#ulCategory > li:nth-child(1) > a:nth-child(4)').get_text()
                    if genre3 == "접기" :
                        genre3 = ""
                    book_list.append([i, month, rank, book_name, author, pub_by, pub_day, price, genre1, genre2, genre3, book_url])
                except :
                    print("오류로 스킵합니다")
                    continue
    
    column_list = ['year', 'month','rank', 'book_name' ,'author', 'pub_by', 'pub_day', 'price', 'genre1', 'genre2', "genre3", "book_url"]
    
    df = pd.DataFrame(np.array(book_list), columns=column_list)
    
    return df

In [ ]:
CategoryNum = 0
year_start = 2001
year_end = 2021
month = 7
page = 2


AladinBookListCrawler(CategoryNum, year_start, year_end, month, page)

In [ ]:
CategoryNum = [798, 170, 987,  336] # 건강/취미/레저X, 경제경영, 과학, 사회과학, 에세이, 인문학, 자기계발
year_start = 2001
year_end = 2021
month = 7
page = 1

for i in CategoryNum:
    df = AladinBookListCrawler(i, year_start, year_end, month, page)
    df.to_csv(f'{year_start}_{year_end}_{i}_{month}월_순위.csv', index=False, encoding='utf-8-sig')